# Setting nodes in a near-Time Loop

## Imports

In [ ]:
# LabOne Q:
from laboneq.simple import *

## Device Setup

In [ ]:
descriptor = """\
instruments:
  HDAWG:
  - address: DEV1001
    uid: device_hdawg
  UHFQA:
  - address: DEV2001
    uid: device_uhfqa
  PQSC:
  - address: DEV3001
    uid: device_pqsc
connections:
  device_hdawg:
    - iq_signal: q0/drive_line
      ports: [SIGOUTS/0, SIGOUTS/1]
    - iq_signal: q1/drive_line
      ports: [SIGOUTS/2, SIGOUTS/3]
    - rf_signal: q0/flux_line
      ports: [SIGOUTS/4]
    - rf_signal: q1/flux_line
      ports: [SIGOUTS/5]
    - to: device_uhfqa
      port: DIOS/0
  device_uhfqa:
    - iq_signal: q0/measure_line
      ports: [SIGOUTS/0, SIGOUTS/1]
    - acquire_signal: q0/acquire_line
    - iq_signal: q1/measure_line
      ports: [SIGOUTS/0, SIGOUTS/1]
    - acquire_signal: q1/acquire_line
  device_pqsc:
    - to: device_hdawg
      port: ZSYNCS/0
"""

## Calibration

In [ ]:
def calibrate_devices(device_setup):
    device_setup.logical_signal_groups["q0"].logical_signals[
        "drive_line"
    ].calibration = SignalCalibration(
        oscillator=Oscillator(
            uid="drive_q0_osc", frequency=1e8, modulation_type=ModulationType.HARDWARE
        ),
        mixer_calibration=MixerCalibration(
            voltage_offsets=[0.02, 0.01],
            correction_matrix=[
                [1.0, 0.0],
                [0.0, 1.0],
            ],
        ),
    )
    device_setup.logical_signal_groups["q0"].logical_signals[
        "flux_line"
    ].oscillator = Oscillator(
        uid="flux_q0_osc", frequency=1e8, modulation_type=ModulationType.HARDWARE
    )
    device_setup.logical_signal_groups["q0"].logical_signals[
        "measure_line"
    ].calibration = SignalCalibration(
        oscillator=Oscillator(
            uid="measure_q0_osc", frequency=1e8, modulation_type=ModulationType.SOFTWARE
        ),
        mixer_calibration=MixerCalibration(
            voltage_offsets=[0.02, 0.01],
            correction_matrix=[
                [1.0, 0.0],
                [0.0, 1.0],
            ],
        ),
    )
    device_setup.logical_signal_groups["q0"].logical_signals[
        "acquire_line"
    ].oscillator = Oscillator(
        uid="acquire_q0_osc", frequency=1e8, modulation_type=ModulationType.SOFTWARE
    )

    device_setup.logical_signal_groups["q1"].logical_signals[
        "drive_line"
    ].calibration = SignalCalibration(
        oscillator=Oscillator(
            uid="drive_q1_osc", frequency=1e8, modulation_type=ModulationType.HARDWARE
        ),
        mixer_calibration=MixerCalibration(
            voltage_offsets=[0.02, 0.01],
            correction_matrix=[
                [1.0, 0.0],
                [0.0, 1.0],
            ],
        ),
    )
    device_setup.logical_signal_groups["q1"].logical_signals[
        "flux_line"
    ].oscillator = Oscillator(
        uid="flux_q1_osc", frequency=1e8, modulation_type=ModulationType.HARDWARE
    )
    device_setup.logical_signal_groups["q1"].logical_signals[
        "measure_line"
    ].calibration = SignalCalibration(
        oscillator=Oscillator(
            uid="measure_q1_osc", frequency=1e8, modulation_type=ModulationType.SOFTWARE
        )
    )
    device_setup.logical_signal_groups["q1"].logical_signals[
        "acquire_line"
    ].oscillator = Oscillator(
        uid="acquire_q1_osc", frequency=1e8, modulation_type=ModulationType.SOFTWARE
    )

### Create Device Setup

In [ ]:
device_setup = DeviceSetup.from_descriptor(
    descriptor,
    server_host="111.22.33.44",
    server_port="8004",
    setup_name="ZI_QCCS",
)
calibrate_devices(device_setup)

## Experiment Definition

In [ ]:
# Near-time loop range
count = 10
start = 0.1
stop = 1.0

x90 = pulse_library.gaussian(uid="x90", length=100e-9, amplitude=1.0)
readout_pulse = pulse_library.const(uid="readout_pulse", length=400e-9, amplitude=1.0)
readout_weighting_function = pulse_library.const(
    uid="readout_weighting_function", length=200e-9, amplitude=1.0
)

# Create Experiment
exp = Experiment(
    uid="User Function",
    signals=[
        ExperimentSignal(uid="q0_drive"),
        ExperimentSignal(uid="q0_measure"),
        ExperimentSignal(uid="q0_acquire"),
    ],
)

q0 = device_setup.logical_signal_groups["q0"]

# Connect experiment signals to logical signals
exp.map_signal("q0_drive", q0.logical_signals["drive_line"])
exp.map_signal("q0_measure", q0.logical_signals["measure_line"])
exp.map_signal("q0_acquire", q0.logical_signals["acquire_line"])

# Apply calibration
q0.logical_signals["drive_line"].calibration.oscillator.frequency = 100e6
q0.logical_signals["measure_line"].calibration.oscillator.frequency = 100e6
q0.logical_signals["drive_line"].calibration.amplitude = 0.5
q0.logical_signals["acquire_line"].calibration.port_delay = 100e-9

average_exponent = 0  # used for 2^n averages, n=average_exponent, maximum: n = 19

sweep_parameter_1 = LinearSweepParameter(uid="amp", start=start, stop=stop, count=count)
sweep_parameter_2 = LinearSweepParameter(uid="reg", start=0, stop=18, count=count)

hdawg_address = device_setup.instrument_by_uid("device_hdawg").address
uhfqa_address = device_setup.instrument_by_uid("device_uhfqa").address

with exp.sweep(uid="sweep", parameter=[sweep_parameter_1, sweep_parameter_2]):
    # Set arbitrary ZI device nodes:
    exp.set_node(path=f"/{hdawg_address}/sigouts/2/offset", value=sweep_parameter_1)
    exp.set_node(path=f"/{hdawg_address}/awgs/0/userregs/0", value=sweep_parameter_2)
    exp.set_node(path=f"/{uhfqa_address}/oscs/0/freq", value=1e6)

    with exp.acquire_loop_rt(
        uid="shots",
        count=pow(2, average_exponent),
        acquisition_type=AcquisitionType.INTEGRATION,
    ):
        with exp.section(uid="qubit_excitation"):
            exp.play(signal="q0_drive", pulse=x90)
        with exp.section(uid="qubit_readout"):
            exp.reserve(signal="q0_drive")
            exp.delay(signal="q0_measure", time=10e-9)
            exp.delay(signal="q0_acquire", time=10e-9)
            exp.play(signal="q0_measure", pulse=readout_pulse)
            exp.acquire(
                signal="q0_acquire",
                handle="q0",
                kernel=readout_weighting_function,
            )
        with exp.section(uid="relax"):
            exp.play(
                signal="q0_measure",
                pulse=pulse_library.const(length=1e-6, amplitude=0.0),
            )  # KNOWN ISSUE: Need to wait for 1us to prevent QA holdoff error

## Execution

In [ ]:
session = Session(device_setup)
session.connect(do_emulation=True)
my_results = session.run(exp)

## Results

In [ ]:
# Nothing specific to the nodes set functionality